In [1]:
'''
  code by Tae Hwan Jung(Jeff Jung) @graykode, modify by wmathor
'''
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

dtype = torch.FloatTensor

In [2]:
sentence = (
    'GitHub Actions makes it easy to automate all your software workflows '
    'from continuous integration and delivery to issue triage and more'
)

word2idx = {w: i for i, w in enumerate(list(set(sentence.split())))}
idx2word = {i: w for i, w in enumerate(list(set(sentence.split())))}
n_class = len(word2idx) # classification problem
max_len = len(sentence.split())
n_hidden = 5

def make_data(sentence):
    input_batch = []
    target_batch = []

    words = sentence.split() # ['Github', 'Actions', 'makes', ...]
    for i in range(max_len - 1): # i = 2
        input = [word2idx[n] for n in words[:(i + 1)]] # input = [18 7 3]
        input = input + [0] * (max_len - len(input)) # input = [18 7 3 0 'it', ..., 0]
        target = word2idx[words[i + 1]] # target = [0]
        input_batch.append(np.eye(n_class)[input])
        target_batch.append(target)

    return torch.Tensor(input_batch), torch.LongTensor(target_batch)

# input_batch: [max_len - 1, max_len, n_class]
input_batch, target_batch = make_data(sentence)
dataset = Data.TensorDataset(input_batch, target_batch)
loader = Data.DataLoader(dataset, 16, True)

In [3]:
class BiLSTM(nn.Module):
    def __init__(self):
        super(BiLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=n_class, hidden_size=n_hidden, bidirectional=True)
        # fc
        self.fc = nn.Linear(n_hidden * 2, n_class)

    def forward(self, X):
        # X: [batch_size, max_len, n_class]
        batch_size = X.shape[0]
        input = X.transpose(0, 1)  # input : [max_len, batch_size, n_class]

        hidden_state = torch.randn(1*2, batch_size, n_hidden)   # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
        cell_state = torch.randn(1*2, batch_size, n_hidden)     # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]

        outputs, (_, _) = self.lstm(input, (hidden_state, cell_state)) # [max_len, batch_size, n_hidden * 2]
        outputs = outputs[-1]  # [batch_size, n_hidden * 2]
        model = self.fc(outputs)  # model : [batch_size, n_class]
        return model

model = BiLSTM()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [4]:
# Training
for epoch in range(10000):
    for x, y in loader:
      pred = model(x)
      loss = criterion(pred, y)
      if (epoch + 1) % 1000 == 0:
          print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

Epoch: 1000 cost = 1.761862
Epoch: 1000 cost = 2.065904
Epoch: 2000 cost = 1.212625
Epoch: 2000 cost = 1.323285
Epoch: 3000 cost = 1.037356
Epoch: 3000 cost = 0.847872
Epoch: 4000 cost = 0.834025
Epoch: 4000 cost = 1.005801
Epoch: 5000 cost = 0.654449
Epoch: 5000 cost = 0.911304
Epoch: 6000 cost = 0.580501
Epoch: 6000 cost = 0.711413
Epoch: 7000 cost = 0.463353
Epoch: 7000 cost = 0.605977
Epoch: 9000 cost = 0.401217
Epoch: 9000 cost = 0.186404
Epoch: 10000 cost = 0.237348
Epoch: 10000 cost = 0.683901


In [6]:
# Pred
predict = model(input_batch).data.max(1, keepdim=True)[1]
print(sentence)
print([idx2word[n.item()] for n in predict.squeeze()])

GitHub Actions makes it easy to automate all your software workflows from continuous integration and delivery to issue triage and more
['it', 'it', 'it', 'easy', 'to', 'automate', 'all', 'your', 'software', 'workflows', 'from', 'continuous', 'integration', 'and', 'delivery', 'to', 'issue', 'and', 'and', 'more']
